In [1]:
import pandas as pd

In [4]:
# Convert relevant files to dataframes
demo = pd.read_sas('../nhanes files/DEMO_L.xpt')
supplements = pd.read_sas('../nhanes files/DSQIDS_L.xpt')
bp = pd.read_sas('../nhanes files/BPXO_L.xpt')
body_measures = pd.read_sas('../nhanes files/BMX_L.xpt')
lab_alb_cr = pd.read_sas('../nhanes files/ALB_CR_L.xpt')
lab_hdl = pd.read_sas('../nhanes files/HDL_L.xpt')
lab_trigly = pd.read_sas('../nhanes files/TRIGLY_L.xpt')
lab_tchol = pd.read_sas('../nhanes files/TCHOL_L.xpt')
lab_glu = pd.read_sas('../nhanes files/GLU_L.xpt')
lab_insulin = pd.read_sas('../nhanes files/INS_L.xpt')
lab_crp = pd.read_sas('../nhanes files/HSCRP_L.xpt')
lab_ferritin = pd.read_sas('../nhanes files/FERTIN_L.xpt')
lab_vitd = pd.read_sas('../nhanes files/VID_L.xpt')

In [14]:
# Demographics
demo_selected = demo[['SEQN', 'RIAGENDR', 'RIDAGEYR', 'RIDRETH3', 'DMDMARTZ', 
                      'DMDEDUC2', 'DMDYRUSR', 'RIDEXPRG', 'INDFMPIR']]

demo_selected.columns = ['SEQN', 'gender', 'age', 'race', 'marital_status',
                         'education', 'years_in_us', 'is_pregnant', 'poverty_index']
demo_selected.head()

,SEQN,gender,age,race,marital_status,education,years_in_us,is_pregnant,poverty_index
0,130378.0,1.0,43.0,6.0,1.0,5.0,6.0,NaN,5.00
1,130379.0,1.0,66.0,3.0,1.0,5.0,NaN,NaN,5.00
2,130380.0,2.0,44.0,2.0,1.0,3.0,6.0,2.0,1.41
3,130381.0,2.0,5.0,7.0,NaN,NaN,NaN,NaN,1.53
4,130382.0,1.0,2.0,3.0,NaN,NaN,NaN,NaN,3.60


In [35]:
# Supplements
supplements_selected = supplements[['SEQN', 'DSQICHOL', 'DSQIFA', 'DSQIVB12', 'DSQIVC', 
                                    'DSQIVD', 'DSQICAFF']]

supplements_selected.columns = ['SEQN', 'takes_cholesterol_supplement', 'takes_folic_acid_supplement',
                                'takes_vitamin_b12_supplement', 'takes_vitamin_c_supplement',
                                'takes_vitamin_d_supplement', 'takes_caffeine_supplement']

# Convert absolute values to binary indicators
supplements_selected[supplements_selected.columns[1:]] = supplements_selected[supplements_selected.columns[1:]].applymap(lambda x: 1 if x>0 else 0)

# Handle duplicate SEQN by taking the maximum value for each supplement
supplements_selected = supplements_selected.groupby('SEQN').max().reset_index()

# Calculate supplement burden score
supplements_selected['supplement_burden_score'] = supplements_selected[supplements_selected.columns[1:]].sum(axis=1)

supplements_selected.head()

C:\Users\UZOMA\AppData\Local\Temp\ipykernel_10332\599126350.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  supplements_selected[supplements_selected.columns[1:]] = supplements_selected[supplements_selected.columns[1:]].applymap(lambda x: 1 if x>0 else 0)
C:\Users\UZOMA\AppData\Local\Temp\ipykernel_10332\599126350.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supplements_selected[supplements_selected.columns[1:]] = supplements_selected[supplements_selected.columns[1:]].applymap(lambda x: 1 if x>0 else 0)


,SEQN,takes_cholesterol_supplement,takes_folic_acid_supplement,takes_vitamin_b12_supplement,takes_vitamin_c_supplement,takes_vitamin_d_supplement,takes_caffeine_supplement,supplement_burden_score
0,130380.0,0,0,0,0,0,0,0
1,130381.0,0,1,1,1,1,0,4
2,130382.0,0,1,1,1,1,0,4
3,130386.0,0,1,1,1,1,0,4
4,130389.0,0,0,0,0,0,0,0


In [16]:
# Blood Pressure
bp_selected = bp[['SEQN', 'BPXOSY1', 'BPXOSY2', 'BPXOSY3',
                  'BPXODI1', 'BPXODI2', 'BPXODI3']]

# Calculate average systolic and diastolic blood pressure, rounding to nearest integer
bp_selected['avg_systolic_bp'] = bp_selected[['BPXOSY1', 'BPXOSY2', 'BPXOSY3']].mean(axis=1).round(0) # skipna=True by default in the mean() function
bp_selected['avg_diastolic_bp'] = bp_selected[['BPXODI1', 'BPXODI2', 'BPXODI3']].mean(axis=1).round(0)

bp_selected = bp_selected[['SEQN', 'avg_systolic_bp', 'avg_diastolic_bp']]
bp_selected.head()

C:\Users\UZOMA\AppData\Local\Temp\ipykernel_10332\1848241566.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bp_selected['avg_systolic_bp'] = bp_selected[['BPXOSY1', 'BPXOSY2', 'BPXOSY3']].mean(axis=1).round(0) # skipna=True by default in the mean() function
C:\Users\UZOMA\AppData\Local\Temp\ipykernel_10332\1848241566.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bp_selected['avg_diastolic_bp'] = bp_selected[['BPXODI1', 'BPXODI2', 'BPXODI3']].mean(axis=1).round(0)


,SEQN,avg_systolic_bp,avg_diastolic_bp
0,130378.0,133.0,96.0
1,130379.0,117.0,79.0
2,130380.0,109.0,78.0
3,130386.0,115.0,74.0
4,130387.0,141.0,76.0


In [ ]:
# Body Anthropometry
body_measures_selected = body_measures[['SEQN', 'BMXWT', 'BMXHT', 'BMXBMI', 
                                        'BMXARMC', 'BMXWAIST', 'BMXHIP']]
body_measures_selected.columns = ['SEQN', 'weight_kg', 'height_cm', 'bmi',
                                  'arm_circumference_cm', 'waist_circumference_cm', 'hip_circumference_cm']
body_measures_selected.head()

,SEQN,weight_kg,height_cm,bmi,arm_circumference_cm,waist_circumference_cm,hip_circumference_cm
0,130378.0,86.9,179.5,27.0,35.7,98.3,102.9
1,130379.0,101.8,174.2,33.5,33.7,114.7,112.4
2,130380.0,69.4,152.9,29.7,36.3,93.5,98.0
3,130381.0,34.3,120.1,23.8,23.4,70.4,NaN
4,130382.0,13.6,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Labs

# Urine albumin to creatinine ratio in mg/g
lab_alb_cr_selected = lab_alb_cr[['SEQN', 'URDACT']]
lab_alb_cr_selected.columns = ['SEQN', 'uacr_mg_per_g']

# HDL cholesterol in mg/dL
lab_hdl_selected = lab_hdl[['SEQN', 'LBDHDD']]
lab_hdl_selected.columns = ['SEQN', 'hdl_cholesterol_mg_per_dL']

# Triglyceride and LDL cholesterol (Martin-Hopkins) in mg/dL
lab_trigly_selected = lab_trigly[['SEQN', 'LBXTLG', 'LBDLDLM']]
lab_trigly_selected.columns = ['SEQN', 'triglycerides_mg_per_dL', 'ldl_cholesterol_martin_hopkins_mg_per_dL']

# Total cholesterol in mg/dL
lab_tchol_selected = lab_tchol[['SEQN', 'LBXTC']]
lab_tchol_selected.columns = ['SEQN', 'total_cholesterol_mg_per_dL']

# Fasting Plasma Glucose in mg/dL
lab_glu_selected = lab_glu[['SEQN', 'LBXGLU']]
lab_glu_selected.columns = ['SEQN', 'fasting_glucose_mg_per_dL']

# Insulin in uU/mL
lab_insulin_selected = lab_insulin[['SEQN', 'LBXIN']]
lab_insulin_selected.columns = ['SEQN', 'insulin_uU_per_mL']

# High-sensitivity C-Reactive Protein in mg/L
lab_crp_selected = lab_crp[['SEQN', 'LBXHSCRP']]
lab_crp_selected.columns = ['SEQN', 'hs_crp_mg_per_L']

# Ferritin in ng/mL
lab_ferritin_selected = lab_ferritin[['SEQN', 'LBXFER']]
lab_ferritin_selected.columns = ['SEQN', 'ferritin_ng_per_mL']

# Vitamin D2 + D3 in nmol/L
lab_vitd_selected = lab_vitd[['SEQN', 'LBXVIDMS']]
lab_vitd_selected.columns = ['SEQN', 'vitamin_d2_d3_nmol_per_L']

In [ ]:
# Combine labs
lab_selected = (
    lab_alb_cr_selected
    .merge(lab_hdl_selected, on='SEQN', how='left')
    .merge(lab_trigly_selected, on='SEQN', how='left')
    .merge(lab_tchol_selected, on='SEQN', how='left')
    .merge(lab_glu_selected, on='SEQN', how='left')
    .merge(lab_insulin_selected, on='SEQN', how='left')
    .merge(lab_crp_selected, on='SEQN', how='left')
    .merge(lab_ferritin_selected, on='SEQN', how='left')
    .merge(lab_vitd_selected, on='SEQN', how='left')
)
lab_selected.head()

,SEQN,uacr_mg_per_g,hdl_cholesterol_mg_per_dL,triglycerides_mg_per_dL,ldl_cholesterol_martin_hopkins_mg_per_dL,total_cholesterol_mg_per_dL,fasting_glucose_mg_per_dL,insulin_uU_per_mL,hs_crp_mg_per_L,ferritin_ng_per_mL,vitamin_d2_d3_nmol_per_L
0,130378.0,17.00,45.0,153.0,190.0,264.0,113.0,15.53,1.78,NaN,58.9
1,130379.0,6.64,60.0,86.0,135.0,214.0,99.0,19.91,2.03,NaN,60.5
2,130380.0,7.92,49.0,375.0,90.0,187.0,156.0,16.33,5.62,13.3,39.4
3,130381.0,7.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,130386.0,4.07,46.0,142.0,111.0,183.0,100.0,11.38,1.05,NaN,96.9


In [36]:
# Merge all selected dataframes into a final dataframe
final_df = (
    demo_selected
    .merge(supplements_selected, on='SEQN', how='left')
    .merge(bp_selected, on='SEQN', how='left')
    .merge(body_measures_selected, on='SEQN', how='left')
    .merge(lab_selected, on='SEQN', how='left')
)
final_df.head()

,SEQN,gender,age,race,marital_status,education,years_in_us,is_pregnant,poverty_index,takes_cholesterol_supplement,...,uacr_mg_per_g,hdl_cholesterol_mg_per_dL,triglycerides_mg_per_dL,ldl_cholesterol_martin_hopkins_mg_per_dL,total_cholesterol_mg_per_dL,fasting_glucose_mg_per_dL,insulin_uU_per_mL,hs_crp_mg_per_L,ferritin_ng_per_mL,vitamin_d2_d3_nmol_per_L
0,130378.0,1.0,43.0,6.0,1.0,5.0,6.0,NaN,5.00,NaN,...,17.00,45.0,153.0,190.0,264.0,113.0,15.53,1.78,NaN,58.9
1,130379.0,1.0,66.0,3.0,1.0,5.0,NaN,NaN,5.00,NaN,...,6.64,60.0,86.0,135.0,214.0,99.0,19.91,2.03,NaN,60.5
2,130380.0,2.0,44.0,2.0,1.0,3.0,6.0,2.0,1.41,0.0,...,7.92,49.0,375.0,90.0,187.0,156.0,16.33,5.62,13.3,39.4
3,130381.0,2.0,5.0,7.0,NaN,NaN,NaN,NaN,1.53,0.0,...,7.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,130382.0,1.0,2.0,3.0,NaN,NaN,NaN,NaN,3.60,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Save final dataframe to CSV
final_df.to_csv('../data/nhanes_final_dataset.csv', index=False)

In [37]:
# Compare the number of rows and columns at each stage
(demo_selected.shape, supplements_selected.shape, bp_selected.shape, body_measures_selected.shape, lab_selected.shape, final_df.shape)

((11933, 9), (4017, 8), (7801, 3), (8860, 7), (8493, 11), (11933, 34))